In [1]:
## This code trained for Ernest Parke! I admit I cannot see a difference between this and what we did together 
## during the CNN tutorial... :) 

## Note that performance on the validation data remains constant, even though loss decreases. This may be a sign
## that the labels on the training and validation are off. Moreover we built a *large* CNN -- it will take many
## epochs to approach good performance! We should write code to check performance on the training data and to check
## loss on the validation data as well. Please see the newest GAN tutorial for code to plot losses via matplotlib.

%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from timeit import default_timer as timer
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

batch_size = 128
data = dsets.ImageFolder("food-101/images", transform=transform)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=6)

# Split our data into training and testing sets
indices = list(range(len(data)))
test_set_size = int(len(data)*0.15)

test_idx = np.random.choice(indices, size=test_set_size, replace=False)
train_idx = list(set(indices) - set(test_idx))

train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(data, batch_size=128, sampler=test_sampler)


In [3]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 5, stride = 1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        ) ## Size: 124x124
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 5, stride = 1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        ) ## Size: 120x120
        
        self.maxpool1 = nn.MaxPool2d(kernel_size = (3, 3), stride = 3)
        ## Size: 40x40
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16)
        ) ## Size: 38x38
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(8)
        ) ## Size: 36x36
        
        self.fc5 = nn.Sequential(
            nn.Linear(in_features=36*36*8, out_features=1000),
            nn.ReLU()
        )
        
        self.fc6 = nn.Sequential(
            nn.Linear(in_features=1000, out_features=101),
            nn.ReLU()
        )
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool1(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = out.view(-1, 36*36*8)
        out = self.fc5(out)
        out = self.fc6(out)
        return out

In [4]:
the_net = cnn()
the_net.cuda()

loss_function = nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = torch.optim.Adam(the_net.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [5]:
num_epochs = 4
train_loss = []
test_loss = []

for epoch in range(num_epochs):
    the_net.train()
    
    for batch_num, (mini_batch_data, mini_batch_labels) in enumerate(train_loader):
        the_batch = Variable(mini_batch_data).cuda()
        the_labels = Variable(mini_batch_labels).cuda()
        
        the_net.zero_grad()
        
        output = the_net(the_batch)
        loss = loss_function(output, the_labels)
        loss.backward()
        optimizer.step()
    
        if batch_num % 50 == 0:
            train_loss.append(loss.detach().cpu().data)
            print('At epoch %i, minibatch %i, Loss: %.4f' % (epoch, batch_num, loss.detach().data))
    
    the_net.eval()
    
    correct = 0
    for _, (test_data, test_labels) in enumerate(test_loader):
        test_labels = Variable(test_labels).cuda()
        output = the_net(Variable(test_data).cuda())
        _, predicted = torch.max(output.data, 1)
        loss = loss_function(output, test_labels)
        test_loss.append(loss.detach().cpu())
        
        correct += float((predicted == test_labels).sum())
        
    print('Accuracy on test: ' + str((1.0*correct)/len(test_idx)))

At epoch 0, minibatch 0, Loss: 4.6209
At epoch 0, minibatch 50, Loss: 4.5452
At epoch 0, minibatch 100, Loss: 4.4411
At epoch 0, minibatch 150, Loss: 4.4452
At epoch 0, minibatch 200, Loss: 4.5808
At epoch 0, minibatch 250, Loss: 4.4001
At epoch 0, minibatch 300, Loss: 4.3312
At epoch 0, minibatch 350, Loss: 4.3666
At epoch 0, minibatch 400, Loss: 4.4522
At epoch 0, minibatch 450, Loss: 4.2434
At epoch 0, minibatch 500, Loss: 4.2962
At epoch 0, minibatch 550, Loss: 4.3392
At epoch 0, minibatch 600, Loss: 4.3416
At epoch 0, minibatch 650, Loss: 4.4307
Accuracy on test: 0.0922112211221122
At epoch 1, minibatch 0, Loss: 4.1994
At epoch 1, minibatch 50, Loss: 4.1923
At epoch 1, minibatch 100, Loss: 4.1449
At epoch 1, minibatch 150, Loss: 3.9528
At epoch 1, minibatch 200, Loss: 4.0883
At epoch 1, minibatch 250, Loss: 4.1903
At epoch 1, minibatch 300, Loss: 4.2866
At epoch 1, minibatch 350, Loss: 4.3403
At epoch 1, minibatch 400, Loss: 3.9578
At epoch 1, minibatch 450, Loss: 4.2457
At epoch 

KeyboardInterrupt: 

In [ ]:
## Great PyTorch tutorial (for v.4) on this as well! https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
alexnet = torchvision.models.alexnet(pretrained=True)
alexnet.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(9216, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 101),
    nn.ReLU()
)

print(alexnet)
alexnet.features.requires_grad = False
alexnet.cuda()

loss_function = nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = torch.optim.Adam(alexnet.parameters(), lr=learning_rate, betas=(0.5, 0.999))